In [1]:
import queue

def process_Intcode(intcode=[99], inputs=queue.Queue(), outputs=queue.Queue(), verbose=True):
    if type(inputs) == list:
        q = inputs
        inputs = queue.Queue()
        [inputs.put(i) for i in q]
    if type(outputs) == list:
        q = outputs
        outputs = queue.Queue()
        [outputs.put(i) for i in q]
    pc = 0
    mem = intcode.copy()
    consumed = 0
    while not (mem[pc] % 100 == 99):
        op = mem[pc] % 100
        modes = []
        for i in range(3):
            modes += [int(mem[pc] / 100 / 10**i) % 10]
        l = 4
        if op == 1:
            A = mem[pc+1] if modes[0] == 1 else mem[mem[pc+1]]
            B = mem[pc+2] if modes[1] == 1 else mem[mem[pc+2]]
            C = mem[pc+3]
            mem[C] = A + B
        elif op == 2:
            A = mem[pc+1] if modes[0] == 1 else mem[mem[pc+1]]
            B = mem[pc+2] if modes[1] == 1 else mem[mem[pc+2]]
            C = mem[pc+3]
            mem[C] = A * B
        elif op == 3:
            A = mem[pc+1]
            mem[A] = inputs.get()
            consumed += 1
            l = 2
        elif op == 4:
            A = mem[pc+1] if modes[0] == 1 else mem[mem[pc+1]]
            outputs.put(A)
            l = 2
        elif op == 5:
            A = mem[pc+1] if modes[0] == 1 else mem[mem[pc+1]]
            B = mem[pc+2] if modes[1] == 1 else mem[mem[pc+2]]
            if A:
                pc = B
                l = 0
            else:
                l = 3
        elif op == 6:
            A = mem[pc+1] if modes[0] == 1 else mem[mem[pc+1]]
            B = mem[pc+2] if modes[1] == 1 else mem[mem[pc+2]]
            if not A:
                pc = B
                l = 0
            else:
                l = 3
        elif op == 7:
            A = mem[pc+1] if modes[0] == 1 else mem[mem[pc+1]]
            B = mem[pc+2] if modes[1] == 1 else mem[mem[pc+2]]
            C = mem[pc+3]
            if A < B:
                mem[C] = 1
            else:
                mem[C] = 0
        elif op == 8:
            A = mem[pc+1] if modes[0] == 1 else mem[mem[pc+1]]
            B = mem[pc+2] if modes[1] == 1 else mem[mem[pc+2]]
            C = mem[pc+3]
            if A == B:
                mem[C] = 1
            else:
                mem[C] = 0
        else:
            return "Something went wrong."
        pc += l
    for i in range(consumed):
        inputs.task_done()
    return mem, list(outputs.queue)

def process_Intcode_test(params, output, message):
    if not process_Intcode(*params) == output:
        print(message)
        return False
    return True

process_Intcode_test([[1,9,10,3,2,3,11,0,99,30,40,50]],
                     ([3500,9,10,70,2,3,11,0,99,30,40,50], []), "FAIL: process_Intcode_test 01")
process_Intcode_test([[1,0,0,0,99]], ([2,0,0,0,99], []), "FAIL: process_Intcode_test 02")
process_Intcode_test([[2,3,0,3,99]], ([2,3,0,6,99], []), "FAIL: process_Intcode_test 03")
process_Intcode_test([[2,4,4,5,99,0]], ([2,4,4,5,99,9801], []), "FAIL: process_Intcode_test 04")
process_Intcode_test([[1,1,1,4,99,5,6,0,99]], ([30,1,1,4,2,5,6,0,99], []), "FAIL: process_Intcode_test 05")
process_Intcode_test([[1002,4,3,4,33]], ([1002,4,3,4,99], []), "FAIL: process_Intcode_test 07")
process_Intcode_test([[1101,100,-1,4,0]], ([1101,100,-1,4,99], []), "FAIL: process_Intcode_test 08")

True

In [2]:
#testing with inputs
process_Intcode_test([[3,0,4,0,99], [456], [], False], ([456,0,4,0,99], [456]), "FAIL: process_Intcode_test 06")
process_Intcode_test([[3,9,8,9,10,9,4,9,99,-1,8], [8], [], False], ([3,9,8,9,10,9,4,9,99,1,8], [1]), "FAIL: process_Intcode_test 09")
process_Intcode_test([[3,9,8,9,10,9,4,9,99,-1,8], [5], [], False], ([3,9,8,9,10,9,4,9,99,0,8], [0]), "FAIL: process_Intcode_test 10")
process_Intcode_test([[3,9,7,9,10,9,4,9,99,-1,8], [5], [], False], ([3,9,7,9,10,9,4,9,99,1,8], [1]), "FAIL: process_Intcode_test 11")
process_Intcode_test([[3,9,7,9,10,9,4,9,99,-1,8], [8], [], False], ([3,9,7,9,10,9,4,9,99,0,8], [0]), "FAIL: process_Intcode_test 12")
process_Intcode_test([[3,3,1108,-1,8,3,4,3,99], [8], [], False], ([3,3,1108,1,8,3,4,3,99], [1]), "FAIL: process_Intcode_test 13")
process_Intcode_test([[3,3,1108,-1,8,3,4,3,99], [5], [], False], ([3,3,1108,0,8,3,4,3,99], [0]), "FAIL: process_Intcode_test 14")
process_Intcode_test([[3,3,1107,-1,8,3,4,3,99], [5], [], False], ([3,3,1107,1,8,3,4,3,99], [1]), "FAIL: process_Intcode_test 15")
process_Intcode_test([[3,3,1107,-1,8,3,4,3,99], [8], [], False], ([3,3,1107,0,8,3,4,3,99], [0]), "FAIL: process_Intcode_test 16")

True

In [3]:
def find_phase_settings(intcode, signal=0, amplifier=0, phases=[]):
    max_signal = signal
    for phase in range(5):
        if phase not in phases:
            sig = process_Intcode(intcode, [phase, signal], [], False)[1][0]
            max_signal = max(max_signal, find_phase_settings(intcode, sig, amplifier+1, phases+[phase]))
    return max_signal

def find_phase_settings_test(params, output, message):
    if not find_phase_settings(*params) == output:
        print(message)
        return False
    return True

find_phase_settings_test([[3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]], 43210, "FAIL: find_phase_settings_test 01")
find_phase_settings_test([[3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]], 54321, "FAIL: find_phase_settings_test 02")
find_phase_settings_test([[3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]], 65210, "FAIL: find_phase_settings_test 03")

True

In [4]:
with open("input07.txt") as infile:
    intcode = [int(i) for i in infile.readlines()[0].split(',')]
find_phase_settings(intcode)

38500

In [5]:
import threading

def permutations(items, taken=[]):
    output = []
    end = True
    for i in items:
        if i not in taken:
            end = False
            output += permutations(items, taken+[i])
    if end:
        return [taken]
    return output

def find_feedback_phase_settings(intcode):
    perms = permutations([i for i in range(5,10)])
    max_sig = 0
    for phases in perms:
        threads = []
        inputs = []
        outputs = []
        for i in range(5):
            inputs.append(queue.Queue())
            outputs.append(queue.Queue())
            t = threading.Thread(target=process_Intcode, kwargs={'intcode': intcode, 'inputs': inputs[i], 'outputs': outputs[i], 'verbose': False})
            t.start()
            threads.append(t)
            inputs[i].put(phases[i])

        signal = 0
        watch = 0
        while threads[watch].is_alive():
            inputs[watch].put(signal)
            signal = outputs[watch].get()
            watch = (watch + 1) % 5

        for t in threads:
            t.join()

        max_sig = max(signal, max_sig)

    return max_sig

def find_feedback_phase_settings_test(params, output, message):
    if not find_feedback_phase_settings(*params) == output:
        print(message)
        return False
    return True

find_feedback_phase_settings_test([[3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]], 139629729, "FAIL: find_feedback_phase_settings_test 01")
find_feedback_phase_settings_test([[3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]], 18216, "FAIL: find_feedback_phase_settings_test 02")

True

In [6]:
with open("input07.txt") as infile:
    intcode = [int(i) for i in infile.readlines()[0].split(',')]
find_feedback_phase_settings(intcode)

33660560